In [ ]:
import chess
import random
import chess.engine
import chess.svg

import keras
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform
import tensorflow as tf 
import pandas as pd


def convertToFeatureVector(boardState):
    pieces = []
    encoding = {'P':1,'R':2,'N':3,'B':4,'Q':5,'K':6,'p':-1,'r':-2,'n':-3,'b':-4,'q':-5,'k':-6, '.':0}
    for i in range(64):
        piece = boardState.piece_at(i)
        if piece:
            piece = piece.symbol()
        else:
            piece = '.'
        pieces.append(encoding[piece])
    return pieces

def evaluateMove(move):
    #simple_dense.h5
    model = tf.keras.models.load_model('simple_dense.h5')
    df = pd.DataFrame(data=[move])
    return model.predict(df)


winCount = 0
printStr = ""
for _ in range(0, 10):

    board = chess.Board()
    engine = chess.engine.SimpleEngine.popen_uci("/usr/local/bin/stockfish")
    nnWhite = True
    if (random.random() < .5):
        result = engine.play(board, chess.engine.Limit(time=0.1))
        board.push(result.move)
        nnWhite = False

    while not board.is_game_over():
        moves = board.legal_moves
        moveStrength = 0
        moveToMake = 0
        for move in moves:
            print("|", end = '')
            board.push(move)
            moveFeature = convertToFeatureVector(board)
            strengthArr = evaluateMove(moveFeature)[0]
            if nnWhite:
                strength = strengthArr[0]
            else:
                strength = strengthArr[1]

            if strength > moveStrength:
                moveToMake = move
                moveStrength = strength
            newMove = board.pop()
        board.push(moveToMake)
        if(board.is_game_over()):
            break;
        result = engine.play(board, chess.engine.Limit(time=0.001))
        board.push(result.move)
        
    display(chess.svg.board(board=board))
    printStr = printStr+""+board.result()+" "+str(nnWhite)+"\n"
    engine.quit()

print(printStr)


||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||